In [1]:
# To get multiple outputs from one code cell (without using print()):
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image, Audio
import sys
from pathlib import Path
import pandas as pd
from pprint import pprint as pp

# For documenting the current environment:
def sys_info():
    frmt = '\nPython ver: {}\nPython env: {}\n'
    frmt += 'OS:         {}\nCurrent dir: {}\n'
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))

# For enabling imports from current project code:
def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = F'Path already in sys.path: {newp}'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = F'Path added to sys.path: {newp}'
    print(msg)

# If this ipynb file is inside a folder, eg ./notebooks, 
# the project code is assumed to reside 1 level up:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), up=Path.cwd().name.startswith(nb_folder))

# For py modules/methods discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
       Example:
       >filter_dir(re) # lists the public methods of the re module.
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

# To create often-used subfolders:
def get_project_dirs(which=['data', 'images'],
                     use_parent=True):
    '''Create folder(s) named in `which` at the ipynb parent level.'''
    if use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    dir_lst = []    
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

# For adding colorfull divider in the nb:
def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)


def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:3em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')

# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2
# .................

Path added to sys.path: C:/Users/catch/Documents/GitHub/DU-event-transcript-demo/resources/EventManagement


In [3]:
import ipywidgets as ipw
from collections import OrderedDict

from manage import (EventMeta as Meta,
                    EventTranscription as TRX,
                    Workflow as FLO,
                    Controls as CTR,
                    Utils as UTL)

main_readme = Meta.MAIN_README

In [28]:
class AppControls:
    def __init__(self):
        self.actions = OrderedDict([('Add an event',
                                     ['Enter Info','Validate','Save','Show']),
                                    ('Modify an event',
                                     ['Modify Event','Validate','Save','Show']),
                                    ('Edit a transcript',
                                     ['Edit Transcript','Save','Show'])
                                   ])
        #save_new, save_existing, save_trx
        self.TR = None
        #self.data_dict = None
        #self.user_dict = None

        # output controls first:
        self.info_out = ipw.Output() # == right-sidebar
        
        self.header = self.get_app_hdr()
        self.left_sidebar_kids_obs = self.menu_tog_sel
        self.left_sidebar = self.get_left()
        self.left_sidebar.observe(self.menu_selection, 'value')
        
        self.center = self.get_center()
        self.center.observe(self.info_display, 'selected_index')
        self.dl1 = ipw.dlink((self.left_sidebar, 'selected_index'),
                             (self.center, 'selected_index'))
        
        self.right_sidebar = self.info_out
        self.footer = None
        self.pane_widths = [1, 5, 1]
        self.pane_heights = [1, 3, 1]

        self.page = ipw.AppLayout(header=self.header,
                                  left_sidebar=self.left_sidebar,
                                  center=self.center,
                                  right_sidebar=self.right_sidebar,
                                  footer=self.footer,
                                  pane_widths=self.pane_widths,
                                  pane_heights=self.pane_heights,
                                  #justify_content='space-between'
                                  )
        setattr(self.page, 'user_dict', None)
        setattr(self.page, 'data_dict', None)


    def get_app_hdr(self):
        style = "text-align:center;padding:5px;background:#c2d3ef;"
        style += "color:#ffffff;font-size:3em;"
        style += "width:100%,height=50%"
        div = F' <div style="{style}">Data Umbrella Event Management</div>'
        hdr_html = FLO.show_du_logo_hdr(as_html=False) + div
        return ipw.HTML(hdr_html)
    
    
    def get_left(self):
        """
        Create Accordion MENU with ToggleButtons children that have:
         - a 'parent_idx' attribute
         - observe function for ToggleButtons children
        """
        acc_items = [ipw.VBox(description=k) for k in list(self.actions.keys())]
        menu_acc = ipw.Accordion(children=acc_items,
                                 selected_index=None)
        for i, (k, v) in enumerate(self.actions.items()):
            btn = ipw.ToggleButtons(options=v, value=None,
                                    button_style='info')
            setattr(btn, 'parent_idx', i)
            btn.observe(self.left_sidebar_kids_obs, names='value')
            menu_acc.children[i].children = [btn]
            menu_acc.set_title(i, k.upper())
        return menu_acc

    
    def get_center(self):
        """ Create 5 Tabs: 3 for actions + 2 for md files."""
        lo_tabs = ipw.Layout(display='flex',
                             flex_flow='column',
                             align_items='stretch',
                             justify_content='center',
                             width='98%')
        tabs = ipw.Tab(selected_index=None, layout=lo_tabs)
        
        # Tabs from menu action + 2 for file display:
        ks = list(self.actions.keys()) #+ ['readme', 'file']
        tabs.children = [ipw.VBox([ipw.Output()]) for k in ks]
        for i, k in enumerate(ks):
            tabs.set_title(i, k.split()[0].upper())

        return tabs


    # left_sidebar.observe
    def menu_selection(change):
        """ 
        Synchronized center (Tabs) selection with 
        left sidebar (menu) selection.
        """
        self.center.selected_index = self.left_sidebar.selected_index


    # center.observe
    def info_display(self, change):
        """Right side panel: info about selected op."""
        self.info_out.clear_output()
        wgt = change['owner']
        # Link tab selection index with info panel:
        if wgt.selected_index is None:
            t = 3
        else:
            t = wgt.selected_index
        if t > 3:
            return
        event_fn = {0:'ADD', 1:'MODIFY', 2:'EDIT', 3:'INIT'}
        which = FLO.EventFunction[event_fn[t]]
        EF = FLO.DisplaySectionInfo(which)
        info_val = EF.show_section_info()
        with self.info_out:
            display(info_val)
   

    def msg_out(self, idx, msg):
        self.center.children[idx].children[0].clear_output()
        with self.center.children[idx].children[0]:
            display(msg)

    
    def validate(self, idx):
        self.center.children[idx].children[0].clear_output()
        
        input_form = self.center.children[idx].children[1]
        if idx == 1:
            self.TR = input_form.TR
            
        with self.center.children[idx].children[0]:
            try:
                self.page.data_dict = FLO.validate_form_entries(input_form,
                                                           self.page.user_dict,
                                                           self.TR)
                print('Validated!')
            except:
                print('Validation Error: Fix & Try again.')
                
    
    def save_entry(self, idx):
        #self.center.children[idx].children[0].clear_output()
        
        input_form = self.center.children[idx].children[1]
        if idx == 1:
            self.TR = input_form.TR
        
        with self.center.children[idx].children[0]:
            if self.page.data_dict is None:
                print('Validate first!')
            return
            try:
                self.TR.update_dict(self.page.data_dict)
                print('Update dict: OK!')
            except:
                print('Update dict: Something went wrong.')
            try:
                self.TR.update_readme()
                print('Update readme: OK!')
            except:
                print('Update readme: Something went wrong.')
            try:
                self.TR.save_transcript_md()
                print('Save: OK!')
            except:
                if idx == 0:
                    msg = 'Save starter transcript: Something went wrong.'
                else:
                    msg = 'Save transcript: Something went wrong.'
                print(msg)


    def save_edit(self, idx):
        """ Save edited transcript. """
        self.center.children[idx].children[0].clear_output()
        
        input_form = self.center.children[idx].children[1]
        self.TR = input_form.TR
        
        with self.center.children[idx].children[0]:
            try:
                self.TR.event_dict['status'] = input_form.status_sel.value
                print('Got status.')
            except:
                print('Could not read Status.')
            try:
                self.TR.update_readme()
                print('Updated README.')
            except:
                print('Could not update README.')
            try:
                self.TR.insert_md_transcript(new_trx=input_form.editarea.value)
                print('Updated Event file.')
            except:
                print('Could not update Event file.')

    
    #def add_file_tab_out2(self, idx):
    #    if len(self.center.children[idx].children) == 1:
    #        self.center.children[idx].children += (ipw.Output(),)
    #    return
        
    def close_file_tabs(self):
        tabs = len(self.center.children)
        if tabs > 3:
            for t in range(3,5):
                self.center.children[t].close()
                
    def add_file_tab(self, idx):
        tabs = len(self.center.children)
        if tabs == 5:
            return
        if tabs == idx:
            title = 'README' if idx == 3 else 'FILE'
            self.center.children[idx] = ipw.VBox([ipw.Output(), ipw.Output()])
            self.center.set_title(idx, title)
        return


    def show_md_file(self, idx):
        """Add tab for Md file. Here, idx = tab index"""
        self.add_file_tab(idx)
        
        self.center.children[idx].children[1].clear_output()
        if idx == 3:
            with self.center.children[3].children[1]:
                Meta.show_md_file(main_readme)
        else:
            if self.TR is not None:
                yr = self.TR.event_dict['year']
                fname = self.TR.event_dict['transcript_md']
                mdfile = Meta.REPO_PATH.joinpath(yr, fname)
                if mdfile.exists():
                    with self.center.children[4].children[1]:
                        Meta.show_md_file(mdfile, kind='Transcript')
                else:
                    self.msg_out(4, F'File not found: {mdfile}.')
            else:
                self.msg_out(4, 'TR object not instanciated.')


    def menu_tog_sel(self, change):
        wgt = change['owner']
        iparent = wgt.parent_idx
        tog_val = wgt.value
                                                
        if tog_val == 'Save':
            if iparent == 0 or iparent == 1:
                self.save_entry(iparent)
            else:
                self.save_edit(iparent)

        elif tog_val == 'Show':
            #if iparent == 0 or iparent == 1:
            self.show_md_file(3) #iparent)
            self.show_md_file(4) #(iparent)
            self.center.selected_index = 3
            
        elif iparent == 0:
            
            if tog_val == 'Enter Info':
                self.close_file_tabs()
                # Instanciate obj for new event:
                self.TR = Meta.TranscriptMeta()
                if Meta.DEMO:
                    # demo data items
                    user_dict = FLO.get_demo_input_dict()  
                else:
                    user_dict = FLO.get_new_input_flds()
                self.page.user_dict = user_dict
                entry_group = FLO.get_entry_accordion(self.page.user_dict)
                # Add 2nd control in tab vbox == input_form:
                self.center.children[0].children += (entry_group,)
                
            elif tog_val == 'Validate':
                self.close_file_tabs()
                self.validate(iparent)

        elif iparent == 1:
            if tog_val == 'Modify Event':
                self.close_file_tabs()
                edit_page = CTR.PageControls(1).page
                # Add 2nd control in tab vbox == input_form:
                self.center.children[1].children += (edit_page, )
                
            elif tog_val == 'Validate':
                self.close_file_tabs()
                self.validate(iparent)

        elif iparent == 2:
            if tog_val == 'Edit Transcript':
                self.close_file_tabs()
                edit_page = CTR.PageControls(2).page
                # Add 2nd control in tab vbox == input_form:
                self.center.children[2].children += (edit_page, )
        return


    def __repr__(self):
        from pprint import pformat
        return pformat(self.__dict__)
    

In [29]:
ap = AppControls()
ap.page

AppLayout(children=(HTML(value='<div style="text-align:center;padding:5px;width:98%"><a href="https://www.data…

In [25]:
i = ap.page.center.selected_index
type(ap.page.center.children[i])
kids = len(ap.page.center.children[i].children)
print('kids:', kids)
type(ap.page.center.children[i].children[0])
if kids > 1:
    type(ap.page.center.children[i].children[1])

ipywidgets.widgets.widget_box.VBox

kids: 2


ipywidgets.widgets.widget_output.Output

ipywidgets.widgets.widget_selectioncontainer.Accordion

In [35]:
len(ap.page.data_dict.keys())

26

In [34]:
ap.TR.event_dict['idn']
#ap.TR.update_dict(ap.page.data_dict)
#ap.TR.update_readme()
#ap.TR.save_transcript_md()

'21'

'Starter transcript created:\n C:\\Users\\catch\\Documents\\GitHub\\DU-event-transcript-demo\\2021\\21-cat-chenal-foo-foo.md'

In [135]:
if len(ap.page.center.children[0].children) == 1:
    # Add an Output as 1st vbx item:
    ap.page.center.children[0].children += (ipw.Output(), )
    
with ap.page.center.children[0].children[1]:
        Meta.show_md_file(main_readme)
    

# idn selection box:

In [99]:
df, empty_last_row, delims = Meta.df_from_readme_tbl()
df.tail(4)

,N,Speaker,Talk Transcript,Transcriber,Status,Notes,year,name
22,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.
23,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.
24,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.
25,23,N.A.,[N.A.](2021/N.A.),?,Not yet processed (editor needed),,2021,N.A.


In [130]:
edit_page = get_edit_page()
edit_page

In [136]:
du_gui

AppLayout(children=(HTML(value='<div style="text-align:center;padding:5px;width:98%"><a href="https://www.data…

In [55]:
form_data_dict = FLO.validate_form_entries(new_entry_group, new_input, TR)
form_data_dict

OrderedDict([('presenter', 'cat chenal'),
             ('title', 'my presentation title'),
             ('event_url', 'N.A.'),
             ('yt_video_id', 'MHAjCcBfT_A'),
             ('slides_url', 'N.A.'),
             ('repo_url', 'https://github.com/CatChenal'),
             ('notebook_url', 'N.A.'),
             ('transcriber', 'N.A.'),
             ('extra_references',
              "- Binder: each listed item should have a 'list header', e.g. '- Binder'  \n- Twitter: Use this format: [full name 1](twitter url), etc.     \n- Wiki: This is an excellent [wiki on transcription](http://en.wikipedia.org/wiki/Main_Page) "),
             ('video_href', 'N.A.'),
             ('video_href_src', 'N.A.'),
             ('video_href_alt', 'N.A.'),
             ('video_href_w', '25%'),
             ('formatted_transcript', 'N.A.'),
             ('year', '2020'),
             ('idn', '22'),
             ('video_url', 'https://www.youtube.com/watch?v=MHAjCcBfT_A'),
             ('title_kw', 'fo

---
---